In [2]:
import pandas as pd
from flask import Flask, render_template, request
import re
import ipywidgets as widgets
from IPython.display import display

In [3]:
df = pd.read_csv("../MiniProjectAPI/data/recipes-clean.csv",encoding='latin1')
df

,vegetarian,vegan,glutenfree,dairyfree,veryhealthy,cheap,aggregatelikes,healthscore,priceperserving,id,...,image,summary,cuisines,dishtypes,diets,analyzedinstructions,spoonacularsourceurl,minutes,calories,protein
0,True,True,True,True,True,False,309,100,134.63,782585,...,https://spoonacular.com/recipeImages/782585-31...,Cannellini Bean and Asparagus Salad with Mushr...,[],"['side dish', 'lunch', 'main course', 'salad',...","['gluten free', 'dairy free', 'lacto ovo veget...","[{'name': '', 'steps': [{'number': 1, 'step': ...",https://spoonacular.com/cannellini-bean-and-as...,45.0,482,31
1,True,True,True,True,True,False,3689,75,112.39,716426,...,https://spoonacular.com/recipeImages/716426-31...,"You can never have too many Chinese recipes, s...","['Chinese', 'Asian']","['side dish', 'antipasti', 'starter', 'snack',...","['gluten free', 'dairy free', 'lacto ovo veget...","[{'name': '', 'steps': [{'number': 1, 'step': ...",https://spoonacular.com/cauliflower-brown-rice...,30.0,192,7
2,True,False,False,False,True,False,689,64,206.79,715497,...,https://spoonacular.com/recipeImages/715497-31...,If you want to add more <b>lacto ovo vegetaria...,[],"['morning meal', 'brunch', 'beverage', 'breakf...",['lacto ovo vegetarian'],"[{'name': '', 'steps': [{'number': 1, 'step': ...",https://spoonacular.com/berry-banana-breakfast...,5.0,457,21
3,False,False,True,True,True,False,1866,100,300.45,715415,...,https://spoonacular.com/recipeImages/715415-31...,Red Lentil Soup with Chicken and Turnips might...,[],"['lunch', 'soup', 'main course', 'main dish', ...","['gluten free', 'dairy free']","[{'name': '', 'steps': [{'number': 1, 'step': ...",https://spoonacular.com/red-lentil-soup-with-c...,55.0,477,27
4,True,True,True,True,True,False,207,100,178.37,716406,...,https://spoonacular.com/recipeImages/716406-31...,Asparagus and Pea Soup: Real Convenience Food ...,[],"['antipasti', 'soup', 'starter', 'snack', 'app...","['gluten free', 'dairy free', 'paleolithic', '...","[{'name': '', 'steps': [{'number': 1, 'step': ...",https://spoonacular.com/asparagus-and-pea-soup...,20.0,217,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,False,False,True,True,True,False,6,83,271.90,975070,...,https://spoonacular.com/recipeImages/975070-31...,Instant Pot Chicken Taco Soup could be just th...,['Mexican'],"['lunch', 'soup', 'main course', 'main dish', ...","['gluten free', 'dairy free']","[{'name': '', 'steps': [{'number': 1, 'step': ...",https://spoonacular.com/instant-pot-chicken-ta...,25.0,346,25
96,True,True,True,True,True,False,2,100,88.42,649886,...,https://spoonacular.com/recipeImages/649886-31...,Lemony Greek Lentil Soup is a Mediterranean ma...,"['Mediterranean', 'European', 'Greek']","['lunch', 'soup', 'main course', 'main dish', ...","['gluten free', 'dairy free', 'lacto ovo veget...","[{'name': '', 'steps': [{'number': 1, 'step': ...",https://spoonacular.com/lemony-greek-lentil-so...,45.0,368,23
97,False,False,True,True,True,False,8,100,1157.26,659927,...,https://spoonacular.com/recipeImages/659927-31...,"Need a <b>gluten free, dairy free, and pescata...",[],"['side dish', 'lunch', 'main course', 'salad',...","['gluten free', 'dairy free', 'pescatarian']","[{'name': '', 'steps': [{'number': 1, 'step': ...",https://spoonacular.com/shrimp-and-avocado-sal...,45.0,639,34
98,True,False,True,True,False,False,295,35,318.98,716217,...,https://spoonacular.com/recipeImages/716217-31...,"Vietnamese Pancakes with Vegetables, Herbs and...","['Vietnamese', 'Asian']",['side dish'],"['gluten free', 'dairy free', 'lacto ovo veget...","[{'name': '', 'steps': [{'number': 1, 'step': ...",https://spoonacular.com/vietnamese-pancakes-wi...,45.0,390,8


In [16]:
def recommend_diet(df, vegetarian=True, vegan=True, dairyfree=True, glutenfree=True, 
                   time_range=None, kcal_range=None, protein_range=None):
    
    # creo filtro en base a las restricciones alimentarias
    if vegetarian:
        df = df[~df['vegetarian']]
    if vegan:
        df = df[~df['vegan']]
    if dairyfree:
        df = df[~df['dairyfree']]
    if glutenfree:
        df = df[~df['glutenfree']]
        
    # creo filtro en base al tiempo necesario
    if time_range:
        if time_range == '0-15':
            df = df[df['minutes'] <= 15]
        elif time_range == '15-30':
            df = df[(df['minutes'] > 15) & (df['minutes'] <= 30)]
        elif time_range == '30-45':
            df = df[(df['minutes'] > 30) & (df['minutes'] <= 45)]
        elif time_range == '45-60':
            df = df[(df['minutes'] > 45) & (df['minutes'] <= 60)]
        elif time_range == '60+':
            df = df[df['minutes'] > 60]
    
    # cre filtro basado en las kcal
    if kcal_range:
        if kcal_range == '0-150':
            df = df[df['calories'] <= 150]
        elif kcal_range == '150-300':
            df = df[(df['calories'] > 150) & (df['calories'] <= 300)]
        elif kcal_range == '300-450':
            df = df[(df['calories'] > 300) & (df['calories'] <= 450)]
        elif kcal_range == '450-600':
            df = df[(df['calories'] > 450) & (df['calories'] <= 600)]
        elif kcal_range == '600-750':
            df = df[(df['calories'] > 600) & (df['calories'] <= 750)]
        elif kcal_range == '750+':
            df = df[df['calories'] > 750]
    
    # creo filtro basado en las proteinas
    if protein_range:
        if protein_range == '0-10':
            df = df[df['protein'] <= 10]
        elif protein_range == '10-20':
            df = df[(df['protein'] > 10) & (df['protein'] <= 20)]
        elif protein_range == '20-30':
            df = df[(df['protein'] > 20) & (df['protein'] <= 30)]
        elif protein_range == '30-40':
            df = df[(df['protein'] > 30) & (df['protein'] <= 40)]
    
    return df


In [18]:
import ipywidgets as widgets
from IPython.display import display

# creamos checkdowns para las alergias
vegetarian = widgets.Checkbox(value=True, description='Vegetarian')
vegan = widgets.Checkbox(value=True, description='Vegan')
dairyfree = widgets.Checkbox(value=True, description='Dairy Free')
glutenfree = widgets.Checkbox(value=True, description='Gluten Free')

# creamos dropdowns para los rangos de valores nutricionales
time_range = widgets.Dropdown(options=['0-15', '15-30', '30-45', '45-60', '60+'], 
                              description='Time Range')
kcal_range = widgets.Dropdown(options=['0-150', '150-300', '300-450', '450-600', '600-750', '750+'], 
                              description='Calories Range')
protein_range = widgets.Dropdown(options=['0-10', '10-20', '20-30', '30-40'], 
                                 description='Protein Range')

# creamos botón para filtrar las recetas
filter_button = widgets.Button(description='Filter Recipes')

# creamos el output
output = widgets.Output()

# define function to handle filter button click
def filter_recipes(button):
    # función para limpiar el output previo
    output.clear_output() 
    with output:
        # aplicar los filtros a las recetas
        filtered_df = recommend_diet(df, vegetarian=vegetarian.value, vegan=vegan.value, 
                                  dairyfree=dairyfree.value, glutenfree=glutenfree.value, 
                                  time_range=time_range.value, kcal_range=kcal_range.value, 
                                  protein_range=protein_range.value)

        # seleccionar solo las columnas que queremos para el output
        filtered_df = filtered_df[['title', 'image', 'spoonacularsourceurl']]

        # mostrar mensaje si no hay recetas que se ajusten a la selección
        if filtered_df.empty:
            print('No recipes found. Please adjust your search criteria.')
        else:
            # mostrar las recetas
            for _, row in filtered_df.iterrows():
                display(widgets.HTML(f'<h3>{row["title"]}</h3><img src="{row["image"]}" /><br><a href="{row["spoonacularsourceurl"]}">{row["spoonacularsourceurl"]}</a>'))

        # limpiar el mensaje en caso de no encontrarse recetas
        if filtered_df.empty and len(output.outputs) > 0:
            output.clear_output()

# añadir botón
filter_button.on_click(filter_recipes)

# display widgets
display(widgets.VBox([widgets.HBox([vegetarian, vegan, dairyfree, glutenfree]),
                      widgets.HBox([time_range, kcal_range, protein_range]),
                      filter_button,
                      output]))
